# Test Agent A2A Walkthrough

This notebook sends a JSON-RPC message to the Test Agent A2A endpoint and prints the response.

Set `APP_HOST` to your app host (without scheme) or to a full URL.

In [7]:
from __future__ import annotations

from uuid import uuid4

import httpx
from databricks.sdk import WorkspaceClient

APP_HOST = "localhost:8000"

if APP_HOST.startswith("http://") or APP_HOST.startswith("https://"):
    BASE_URL = APP_HOST.rstrip("/")
else:
    BASE_URL = f"http://{APP_HOST}".rstrip("/")

if "localhost" in APP_HOST:
    headers: dict[str, str] = {}
else:
    w = WorkspaceClient()
    headers = w.config.authenticate()

A2A_URL = f"{BASE_URL}/test-agent/"
print("A2A_URL:", A2A_URL)

A2A_URL: http://localhost:8000/test-agent/


In [8]:
def send_test_agent_message(text: str) -> dict:
    payload = {
        "jsonrpc": "2.0",
        "id": uuid4().hex,
        "method": "message/send",
        "params": {
            "message": {
                "role": "user",
                "parts": [{"kind": "text", "text": text}],
                "message_id": uuid4().hex,
            }
        },
    }
    resp = httpx.post(A2A_URL, json=payload, headers=headers, timeout=60.0)
    resp.raise_for_status()
    return resp.json()

In [11]:
response = send_test_agent_message("hello from notebook")
print(response)

{'id': '1fe50cd16224484c9d6276265aed942c', 'jsonrpc': '2.0', 'result': {'artifacts': [{'artifactId': 'b3a7990c-6c97-4dc4-90bd-68ac384404a8', 'name': 'test_agent_response', 'parts': [{'kind': 'text', 'text': '{\n  "message": "hello world from Test Agent",\n  "handshake": {\n    "context_id": "7feaca9a-c0ac-44df-857f-f9321b79dbaa",\n    "task_id": "e66b0230-c457-4f06-a5a8-3a6439e49215",\n    "message_id": "96d2628842a54f88a173b2921466ee1c"\n  },\n  "request": {\n    "input_text": "hello from notebook"\n  },\n  "history_size": 5\n}'}]}], 'contextId': '7feaca9a-c0ac-44df-857f-f9321b79dbaa', 'history': [{'contextId': '7feaca9a-c0ac-44df-857f-f9321b79dbaa', 'kind': 'message', 'messageId': '96d2628842a54f88a173b2921466ee1c', 'parts': [{'kind': 'text', 'text': 'hello from notebook'}], 'role': 'user', 'taskId': 'e66b0230-c457-4f06-a5a8-3a6439e49215'}], 'id': 'e66b0230-c457-4f06-a5a8-3a6439e49215', 'kind': 'task', 'status': {'state': 'completed', 'timestamp': '2026-02-09T04:51:31.709988+00:00'}}